In [4]:
import pandas as pd 
df = pd.read_csv('amazon_earbuds.csv')
df.drop_duplicates(subset=['Asin'], inplace = True) # now we have 250  rows instead of 381 as at the beginning
data_asin = list(df.Asin)
data_asin[0:5]

['B08LGRK6MT', 'B08FB2V4GR', 'B08C5HYHYB', 'B08KDZ2NZX', 'B08832CDT9']

In [ ]:
import requests
from bs4 import BeautifulSoup

getfile = 'reviews.csv'
#Now create link to product with ASIN and get the link to page contains all customer reviews
def get_review(asin):
    url="https://www.amazon.de/dp/"+asin #get product link
    header = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Mobile Safari/537.36 Edg/95.0.1020.44'}
    r = requests.get(url, headers = header) #down page
    content = r.content
    soup=BeautifulSoup(content,"html.parser")
    #search link to page contains all customer reviews
    i = soup.find('a',{'data-hook':'see-all-reviews-link-mobile'})
        #search and append reviews to dataFrame
    link = i['href']
    #search reviews on all pages
    review_data = pd.DataFrame(columns=['Rating', 'Review Title','Review Content'])
    for i in range(50):
        url_review ='https://www.amazon.de'+link+'&pageNumber='+str(i)
        re = requests.get(url, headers = header)
        content_r = re.content
        soup_r=BeautifulSoup(content_r,"html.parser")
        for j in soup_r.find_all('div',{'class':'a-row a-spacing-micro a-grid-vertical-align a-grid-center'}):
            ratings = j.find_all('i',{'data-hook':'review-star-rating'})
            titles = j.find_all('span',{'data-hook':'review-title'})
            review_contents = j.find_all('span',{'class':'a-size-base reviewText review-text-content'})
            for element in ratings:
                if element is not None:
                    rating = element.text
                else:
                    rating = 'unknown'
            for element in titles:
                if element is not None:
                    title = element.text
                else:
                    title = 'unknown' 
            for element in review_contents:
                if element is not None:
                    review_content = element.text
                else:
                    review_content = 'unknown'
            review_data = review_data.append({'Rating':rating, 'Review Title':title,'Review Content':review_content},ignore_index=True)
    return review_data
result = pd.DataFrame()
for i in range(len(data_asin)):
    df = get_review(data_asin[i])
    result = result.append(df)
result.to_csv(getfile, index=False, encoding='utf-8')

In [10]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
url='https://www.amazon.de/JBL-Reflect-Mini-Ear-Sport-Kopfh%C3%B6rer-Noise-Cancelling-Schwarz/product-reviews/B08FB2V4GR/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&amp%3BreviewerType=all_reviews' #get product link
header = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Mobile Safari/537.36 Edg/95.0.1020.44'}
r = requests.get(url, headers = header) #down page
content = r.content
soup=BeautifulSoup(content,"html.parser")
for j in soup.find_all('div',{'class':'a-row a-spacing-micro a-grid-vertical-align a-grid-center'}):
    ratings = j.find_all('i',{'data-hook':'review-star-rating'})
    titles = j.find_all('span',{'data-hook':'review-title'})
    review_contents = j.find_all('span',{'class':'a-size-base reviewText review-text-content'})
    print(ratings)


<!DOCTYPE html>

<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.de</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-eu.amazon.de",
        ue_mid = "A1PA6795UKMFR9",